In [ ]:
%load_ext autoreload
%autoreload 2
import os
from v4vapp_backend_v2.config.setup import InternalConfig, logger
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults
from nectar.amount import Amount

from google.protobuf.json_format import MessageToDict

import v4vapp_backend_v2.lnd_grpc.lightning_pb2 as lnrpc
from v4vapp_backend_v2.lnd_grpc.lnd_client import LNDClient

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())

CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()


async def add_invoice(
    value_msat: int, memo: str, connection_name: str = "umbrel"
) -> lnrpc.AddInvoiceResponse:
    # https://lightning.engineering/api-docs/api/lnd/router/add-invoice/
    async with LNDClient(connection_name=connection_name) as client:
        request = lnrpc.Invoice(value_msat=value_msat, memo=memo)
        response: lnrpc.AddInvoiceResponse = await client.lightning_stub.AddInvoice(request)
        add_invoice_response_dict = MessageToDict(response, preserving_proto_field_name=True)
        logger.info(
            f"Invoice generated: {memo} {value_msat // 1000:,} sats",
            extra={"add_invoice_response_dict": add_invoice_response_dict},
        )
        return response

In [ ]:
from v4vapp_backend_v2.config.setup import HiveRoles
from v4vapp_backend_v2.actions.hive_to_lightning import get_verified_hive_client
from nectar.account import Account
from v4vapp_backend_v2.helpers.crypto_conversion import CryptoConversion
from v4vapp_backend_v2.helpers.crypto_prices import Currency
from v4vapp_backend_v2.hive.hive_extras import send_transfer
from pprint import pprint


hive_config = InternalConfig().config.hive
hive_client, customer = await get_verified_hive_client(hive_role=HiveRoles.customer)
server = hive_config.get_hive_role_account(hive_role=HiveRoles.server).name
connection = "umbrel"
send_sats = 11_000

send_conv = CryptoConversion(
    conv_from=Currency.SATS,
    value=send_sats,
)
await send_conv.get_quote()
conv = send_conv.conversion

invoice = await add_invoice(
    value_msat=send_sats * 1_000,  # 1,000,000 msats
    memo="Test Invoice for 1,000,000 msats",
    connection_name=connection,
)
print("Invoice created:", invoice)

amount_to_send_msats = conv.msats + conv.msats_fee + 200_0000  # Adding a buffer for fees
amount_to_send_hive = (amount_to_send_msats // 1000) / conv.sats_hive
hive_amount = Amount(f"{amount_to_send_hive:.3f} HIVE")

trx = await send_transfer(
    from_account=customer,
    to_account=server,
    hive_client=hive_client,
    amount=hive_amount,
    memo=f"{invoice.payment_request}",
)
